In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import kagglehub
from kagglehub import KaggleDatasetAdapter
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

Assumptions made 
1. Use recidivism in favor of violent_recidivism 

In [ ]:
import os
filename = "compas-scores.csv"
assert(filename in os.listdir())

original_columns =  ['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'num_r_cases',
       'r_case_number', 'r_charge_degree', 'r_days_from_arrest',
       'r_offense_date', 'r_charge_desc', 'r_jail_in', 'r_jail_out',
       'is_violent_recid', 'num_vr_cases', 'vr_case_number',
       'vr_charge_degree', 'vr_offense_date', 'vr_charge_desc',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'type_of_assessment', 'decile_score.1',
       'score_text', 'screening_date']

date_cols = ["c_jail_in", "c_jail_out", "r_jail_in", "r_jail_out", "r_offense_date"]

columns = [ 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 
       'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 
       'r_charge_degree', 'r_days_from_arrest',
       'r_offense_date', 'r_charge_desc', 'r_jail_in', 'r_jail_out',
       'is_violent_recid', 'num_vr_cases',
       'vr_charge_degree', 'vr_charge_desc',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'type_of_assessment', 'decile_score.1',
       'score_text']

csv = pd.read_csv(filename, usecols=columns, parse_dates=date_cols)
csv = csv[csv["is_recid"] != -1] 

In [ ]:
np.random.seed(40)

# drop all is_recid = -1 since that indicates a problem with the data

csv["current_jailtime"] = (csv["c_jail_out"] - csv["c_jail_in"]).dt.total_seconds().div(84000).fillna(0) 
csv["previous_jailtime"] = (csv["r_jail_out"] - csv["r_jail_in"]).dt.total_seconds().div(84000).fillna(0) 
csv.drop(labels=["r_jail_in", "r_jail_out", "c_jail_in", "c_jail_out"], axis=1)

(11038, 32)


,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,c_charge_degree,...,vr_charge_degree,vr_charge_desc,v_type_of_assessment,v_decile_score,v_score_text,type_of_assessment,decile_score.1,score_text,current_jailtime,previous_jailtime
0,Greater than 45,Other,0,1,0,0,0,-1.0,1.0,F,...,NaN,NaN,Risk of Violence,1,Low,Risk of Recidivism,1,Low,1.012595,0.0
2,25 - 45,African-American,0,3,0,0,0,-1.0,1.0,F,...,(F3),Felony Battery (Dom Strang),Risk of Violence,1,Low,Risk of Recidivism,3,Low,10.365310,0.0
3,Less than 25,African-American,0,4,0,1,4,-1.0,1.0,F,...,NaN,NaN,Risk of Violence,3,Low,Risk of Recidivism,4,Low,1.116786,0.0
4,Less than 25,African-American,0,8,1,0,1,NaN,1.0,F,...,NaN,NaN,Risk of Violence,6,Medium,Risk of Recidivism,8,High,0.000000,0.0
5,25 - 45,Other,0,1,0,0,2,NaN,76.0,F,...,NaN,NaN,Risk of Violence,1,Low,Risk of Recidivism,1,Low,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11752,Greater than 45,Other,0,3,0,0,1,-1.0,1.0,F,...,NaN,NaN,Risk of Violence,1,Low,Risk of Recidivism,3,Low,1.207298,0.0
11753,Less than 25,Caucasian,0,7,3,5,3,-1.0,1.0,M,...,NaN,NaN,Risk of Violence,5,Medium,Risk of Recidivism,7,Medium,2.014940,0.0
11754,25 - 45,Other,0,4,0,0,0,NaN,1.0,M,...,NaN,NaN,Risk of Violence,3,Low,Risk of Recidivism,4,Low,0.000000,0.0
11755,25 - 45,Caucasian,0,3,0,0,2,-196.0,196.0,M,...,NaN,NaN,Risk of Violence,2,Low,Risk of Recidivism,3,Low,1.279548,0.0
